# renewable data from REPD

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import re
import os
os.chdir('/Users/simon/Desktop/uk-map/')
%matplotlib inline

In [24]:
# import the REPD

df = pd.read_excel('repd-q1-2019.xlsx', sheetname='Database') 

# force dates to be dates
df.convert_objects(convert_dates='coerce', convert_numeric=True)


# set column names to the 5th row
df.columns = df.iloc[4]
df.drop(df.index[0:5],inplace=True)

# drop rows without data
df.dropna(subset=['Site Name'], inplace=True)

# get rid of commas in the X-Y coordinates else it breaks the X-Y to lat-lon code

#df['X-coordinate'] = df['X-coordinate'].str.replace(',','')
#df['Y-coordinate'] = df['Y-coordinate'].str.replace(',','')


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [25]:
# suppress errors in the "for" loops (I didn't understand them and they didn't seem to matter)
pd.options.mode.chained_assignment = None  # default='warn'

In [26]:
#df.head()

In [27]:
# drop entries without X-Y
# there aren't that many


df.dropna(subset=['X-coordinate'],inplace=True)

In [28]:

# drop non-data rows and columns

df.drop(df.index[:5], inplace=True)
df.columns.values[0] = 'blank1'
df.columns.values[1] = 'blank2'

# drop first two columns that are blank
df.drop(['blank1','blank2'], axis=1, inplace=True)


In [29]:
# fill empty x and y coords with locations as follows:
#df['X-coordinate'] = df['X-coordinate'].fillna(-477441) # somewhere west of Ireland
#df['Y-coordinate'] = df['Y-coordinate'].fillna(481803)

# there is about 45MW of no-location operational capacity
# some of it is eg Sainsburys solar rooftops across multiple sites but listed together
# might get caught by FiTs database, depending how we handle that

# correct entry with a comma in it
#df['X-coordinate'].loc[5178] = 51087

In [30]:
# convert X-Y coordinates to lat-lon

def proj_transform(df):
    bng = pyproj.Proj(init='epsg:27700')
    wgs84 = pyproj.Proj(init='epsg:4326')
    lon,lat = pyproj.transform(bng,wgs84, df['X-coordinate'], df['Y-coordinate'])    
    df['lat'] = lat
    df['lon'] = lon
    return df

df = df.apply(proj_transform, axis=1)

In [31]:
# select operational plants

operational = df.loc[df['Development Status']=='Operational']

# convert to datetime

operational['Operational'] =  pd.to_datetime(operational['Operational'])

# add year it opened

operational['Year Open'] = operational['Operational'].dt.year

In [32]:
#operational.head()

In [33]:
# select the columns we want
operational = operational[[
    'Site Name', 'Technology Type', 'Installed Capacity (MWelec)', 'Region',
    'lat', 'lon', 'Year Open','CHP Enabled','Operator (or Applicant)'
]]

In [34]:
# map the tech types into fuel/type as per conventional plants

di = {
    'Biomass (dedicated)': 'Biomass',
    'Advanced Conversion Technologies': 'Waste',
    'Anaerobic Digestion': 'Waste',
    'EfW Incineration': 'Waste',
    'Biomass (co-firing)': 'Biomass',
    'Large Hydro': 'Hydro',
    'Small Hydro': 'Hydro',
    'Landfill Gas': 'Waste',
    'Solar Photovoltaics': 'Solar',
    'Sewage Sludge Digestion' : 'Waste',
    'Tidal Barrage and Tidal Stream' : 'Wave & Tidal',
    'Shoreline Wave' : 'Wave & Tidal',
    'Wind Offshore' : 'Wind',
    'Wind Onshore' : 'Wind',
    'Pumped Storage Hydroelectricity' : 'Storage',
    'Battery' : 'Storage',
    'Flywheels' : 'Storage'
}

operational['Fuel'] = operational['Technology Type'].map(di)


di2 = {
    
    'Advanced Conversion Technologies': 'Advanced Conversion',
    'Anaerobic Digestion': 'Anaerobic Digestion',
    'EfW Incineration': 'Waste Incinerator',
    'Biomass (co-firing)': 'Co-firing',
    'Large Hydro': 'Large Hydro',
    'Small Hydro': 'Small Hydro',
    'Landfill Gas': 'Landfill Gas',
    'Sewage Sludge Digestion' : 'Sewage Sludge Digestion',
    'Tidal Barrage and Tidal Stream' : 'Tidal',
    'Shoreline Wave' : 'Wave',
    'Wind Offshore' : 'Offshore Wind',
    'Wind Onshore' : 'Onshore Wind',
    'Pumped Storage Hydroelectricity' : 'Pumped Hydro',
    'Battery' : 'Battery',
    'Flywheels' : 'Flywheeel'
}

operational['Type'] = operational['Technology Type'].map(di2)

In [35]:
# Add yearStart and yearEnd columns

# if the site opened before 2004, yearStart is 2004; otherwise opening year

operational['yearStart'] = np.where(operational['Year Open']<2004,2004,operational['Year Open'])

# sites continue to operate until 2030 unless they're wind/solar that would be >25 by then

operational['yearEnd'] = np.where(((operational['Year Open']<2005)&((operational.Fuel=='Wind')|(operational.Fuel=='Solar'))),2018,2019)

In [36]:
# drop tech type and rename other columns

operational = operational[[
    'Site Name', 'Installed Capacity (MWelec)', 'Region', 'lat', 'lon',
    'Year Open', 'CHP Enabled', 'Operator (or Applicant)', 'Fuel', 'Type',
    'yearStart', 'yearEnd'
]]

operational.columns = [
    'Site', 'Capacity (MW)', 'Region', 'Latitude', 'Longitude', 'Year Open',
    'CHP', 'Company', 'Fuel', 'Type', 'yearStart', 'yearEnd'
]

In [37]:
# group Drax units

operational.Site[operational.Site.str.contains('Drax')] = 'Drax Biomass'
operational.yearEnd[(operational.Site == 'Drax')&(operational.yearStart == 2013)] = 2013
operational.yearEnd[(operational.Site == 'Drax')&(operational.yearStart == 2014)] = 2016
operational.yearEnd[(operational.Site == 'Drax')&(operational.yearStart == 2017)] = 2017

operational['Capacity (MW)'][(operational.Site == 'Drax')&(operational.yearStart == 2014)] = 645*2
operational['Capacity (MW)'][(operational.Site == 'Drax')&(operational.yearStart == 2017)] = 645*3
operational['Capacity (MW)'][(operational.Site == 'Drax')&(operational.yearStart == 2018)] = 645*4


In [38]:
# add future 

# look at stuff under construction

building = df[df['Development Status']=='Under Construction']

# select the columns we want

building = building[[
    'Site Name', 'Technology Type', 'Installed Capacity (MWelec)', 'Region',
    'lat', 'lon', 'CHP Enabled','Operator (or Applicant)'
]]

building['yearStart'] = 2019
building['yearEnd'] = 2019

# use dicts from above to map tech types again

building['Fuel'] = building['Technology Type'].map(di)
building['Type'] = building['Technology Type'].map(di2)

# drop tech type and rename other columns

building = building[[
    'Site Name', 'Installed Capacity (MWelec)', 'Region', 'lat', 'lon',
    'CHP Enabled', 'Operator (or Applicant)', 'Fuel', 'Type',
    'yearStart', 'yearEnd'
]]

building.columns = [
    'Site', 'Capacity (MW)', 'Region', 'Latitude', 'Longitude',
    'CHP', 'Company', 'Fuel', 'Type', 'yearStart', 'yearEnd'
]


In [39]:
# add sites that already have CfDs but are not yet under construction

# sites drawn from CfD register as not terminated
# not yet operational nor under construction

di3 = {
    'BHEG Walsall': 'Yes',
    'Blackbridge TGS 1':'Yes',
    'Hornsea Project Two - Optimus and Breesea': 'Yes',
    'Fort Industrial Park (Castle Bromwich)': 'Yes',
    'MacColl Offshore Windfarm (Moray Firth)':'Yes',
    'Telford Offshore Windfarm (Moray Firth)':'Yes',
    'Stevenson Offshore Windfarm (Moray Firth)':'Yes',
    'Neart na Gaoithe':'Yes',
    'Northacre RRC':'Yes',
    'Solwaybank Wind Farm (resubmission)':'Yes',
    'Triton Knoll':'Yes',
    'Wren Power and Pulp (Rivenhall Airfield)':'Yes'
}

df['Contracted'] = df['Site Name'].map(di3)

cfd = df[df.Contracted=='Yes']

cfd['yearStart'] = 2019
cfd['yearEnd'] = 2019

# use dicts from above to map tech types again

cfd['Fuel'] = cfd['Technology Type'].map(di)
cfd['Type'] = cfd['Technology Type'].map(di2)

# drop tech type and rename other columns

cfd = cfd[[
    'Site Name', 'Installed Capacity (MWelec)', 'Region', 'lat', 'lon',
    'CHP Enabled', 'Operator (or Applicant)', 'Fuel', 'Type',
    'yearStart', 'yearEnd'
]]

cfd.columns = [
    'Site', 'Capacity (MW)', 'Region', 'Latitude', 'Longitude',
    'CHP', 'Company', 'Fuel', 'Type', 'yearStart', 'yearEnd'
]


In [40]:
# append "building" and "cfd" to the operational plants

renewable = operational.append(building)
renewable = renewable.append(cfd)




In [41]:
# remove Drax and Lynemouth from this data as fixed in conv

# make sure the index is unique before dropping rows
renewable = renewable.reset_index(drop=True)

renewable = renewable.drop(renewable[(
    (renewable.Site == 'Drax Biomass'))].index)

renewable = renewable.drop(renewable[(
    (renewable.Site == 'Lynemouth Power Station'))].index)

# remove Uskmouth co-firing

renewable = renewable.drop(renewable[(
    (renewable.Site == 'Uskmouth Power Station'))].index)


In [42]:


# get rid of rogue line breaks and carriage returns in the company names

renewable['Company'] = renewable['Company'].str.replace('\n', ' ')
renewable['Company'] = renewable['Company'].str.replace('\r', ' ')

# save the output

renewable.to_csv('renewable.csv',index=False,encoding='utf-8')


# analysis and checks


In [170]:
renewable[renewable.Site.str.contains('Drax')]

,CHP,Capacity (MW),Company,Fuel,Latitude,Longitude,Region,Site,Type,Year Open,yearEnd,yearStart


In [147]:
renewable[renewable.yearEnd==2018]

,CHP,Capacity (MW),Company,Fuel,Latitude,Longitude,Region,Site,Type,Year Open,yearEnd,yearStart
2019,n/a,4.000,Blyth Harbor Wind Ltd,Wind,55.136119,-1.490068,Offshore,Blyth Offshore,Offshore Wind,2000.0,2018,2004
2027,n/a,60.000,Greencoat (formerly Npower Renewables),Wind,53.417246,-3.448574,Offshore,North Hoyle,Offshore Wind,2003.0,2018,2004
2031,n/a,60.000,E.On,Wind,52.643946,1.788903,Offshore,Scroby Sands,Offshore Wind,2004.0,2018,2004
2077,n/a,1.470,Ecotricity,Wind,52.709894,1.654462,Eastern,South Beach Wind Turbine (Somerton),Onshore Wind,2000.0,2018,2004
2078,n/a,1.470,LRZ Ltd (Taken over by NOVERA),Wind,52.656729,0.683267,Eastern,Swaffham 1 - Ecotech Wind Park,Onshore Wind,1999.0,2018,2004
2079,n/a,10.200,Cynllun Trydan Gwynt,Wind,52.466297,-3.971001,Wales,Mynydd Gorddu Wind Farm,Onshore Wind,1998.0,2018,2004
2083,n/a,16.800,RWE npower,Wind,52.549433,-3.608163,Wales,Carno 'B' Wind Farm,Onshore Wind,1996.0,2018,2004
2084,n/a,16.800,RWE npower,Wind,52.542306,-3.579191,Wales,Carno 'A' Wind Farm,Onshore Wind,1996.0,2018,2004
2085,n/a,3.400,Vestas,Wind,52.657911,-3.672068,Wales,Cemmaes 'B' Wind Farm,Onshore Wind,2002.0,2018,2004
2086,n/a,5.600,RWE npower,Wind,53.374760,-4.346466,Wales,Trysglwyn Wind Farm,Onshore Wind,1996.0,2018,2004


In [127]:
renewable.Site[renewable.yearStart==2019]

92                            Port Clarence Biomass Plant
95                Merevale & Blyth Estate (Biomass Plant)
101                   Enviroparks Hirwaun Generation Site
133                           Tees Renewable Energy Plant
141           Brite Partnership Biomass Plant / Rotherham
144                            Pulse Park, Wellingborough
150                        Charlesfield Biomass CHP Plant
152                                        Discovery Park
261                 South Kirkby Business Park (Waste AD)
272     Queen Charlton Quarry Renewable Energy Facilit...
320                                                      
332                     Charlton Lane Eco Park (Waste AD)
346                                    Knostrop WWTW AD -
807         Ratty's Lane Sustainable Enery Facility (ACT)
827     Glasgow Renewable Energy and Recycling Centre ...
841                                        Beddington EfW
844                         K3 CHP Facility (Kemsley EfW)
854           

In [121]:
cfd

,Site,Capacity (MW),Region,Latitude,Longitude,CHP,Company,Fuel,Type,yearStart,yearEnd
875,Wren Power and Pulp (Rivenhall Airfield),49.75,Eastern,51.853283,0.647025,Yes,Gent Fairhead Environmental Services,Waste,Waste Incinerator,2019,2019
884,BHEG Walsall,27.00,West Midlands,52.610755,-2.009556,No,BH EnergyGap (Walsall),Waste,Advanced Conversion,2019,2019
2034,Hornsea Project Two - Optimus and Breesea,1400.00,Offshore,53.959998,1.539986,n/a,Orsted (formerly Dong Energy),Wind,Offshore Wind,2019,2019
2052,Neart na Gaoithe,450.00,Offshore,56.270000,-2.250001,n/a,EDF Renewables Power,Wind,Offshore Wind,2019,2019
2062,MacColl Offshore Windfarm (Moray Firth),317.00,Offshore,57.967325,-2.790537,n/a,Moray East (formerly MORL)/ EDPR,Wind,Offshore Wind,2019,2019
2064,Telford Offshore Windfarm (Moray Firth),317.00,Offshore,58.211232,-2.790540,n/a,Moray East (formerly MORL)/ EDPR,Wind,Offshore Wind,2019,2019
2065,Stevenson Offshore Windfarm (Moray Firth),316.00,Offshore,58.054625,-3.158579,n/a,Moray East (formerly MORL)/ EDPR,Wind,Offshore Wind,2019,2019
3631,Solwaybank Wind Farm (resubmission),30.00,Scotland,55.100174,-3.107751,n/a,The Renewables Infrastructure Group (TRIG),Wind,Onshore Wind,2019,2019
4546,Northacre RRC,22.00,South West,51.271237,-2.209908,Yes,Northacre Renewable Energy (Hills Group),Waste,Advanced Conversion,2019,2019
4792,Fort Industrial Park (Castle Bromwich),10.20,West Midlands,52.511827,-1.801317,No,Industrial Property Investment Fund,Waste,Advanced Conversion,2019,2019


In [116]:
df[['Site Name', 'Installed Capacity (MWelec)',
    'Development Status (short)','Technology Type']][(df['Site Name'].str.contains('Wren'))]

4,Site Name,Installed Capacity (MWelec),Development Status (short),Technology Type
875,Wren Power and Pulp (Rivenhall Airfield),49.75,Awaiting Construction,EfW Incineration


In [78]:
#df[(df['Technology Type']=='Wind Onshore')&(df['Development Status (short)']=='Operational')].sort_values(by='Installed Capacity (MWelec)',ascending=False)

In [60]:
# pull CfD register for reference

reg = pd.read_excel('CfD Register as at 25-01-19.xlsx', sheetname='CfD Register as at 250119') 

reg.columns = [
    'Name','ID','Start','Last update','Round','Tech','CHP','Dual?','Whole MW','Capacity (MW)','a','b','c','d','Target date','Window','Expected date','Operator','Country','Address 1','Address 2','Town/City','County','Postcode','Registration','Coordinates','Agreement type','Terms','Ref no','Connection type','Connection','Licence'
,'Metering','Leasing round','Termination date','Comment']

reg = reg[['Name','Tech','CHP','Capacity (MW)','Target date','Expected date','Operator','Coordinates','Termination date']]

reg.sort_values(by='Name',inplace=True)

In [61]:
# list of sites on the CfD register

reg.Name[reg['Termination date'].isnull()].tolist()

[u'Achlachan Wind Farm',
 u'BHEG Walsall',
 u'Bad a Cheo Wind Farm',
 u'Beatrice Phase 1',
 u'Beatrice Phase 2',
 u'Blackbridge TGS 1 Limited',
 u'Brenig Wind Farm - Brenig Wind Limited',
 u'Burbo',
 u'Charity Farm',
 u'Clocaenog Forest Wind Farm',
 u'Coire Na Cloiche Windfarm',
 u'Common Barn Wind Farm',
 u'Dorenell Wind Farm',
 u'Drax 3rd Conversion Unit (Unit 1)',
 u'Dudgeon Phase 1',
 u'Dudgeon Phase 2',
 u'Dudgeon Phase 3',
 u'EA 1, Phase 1',
 u'EA 1, Phase 2',
 u'EA 1, Phase 3',
 u'Energy Works (Hull)',
 u'Enviroparks Hirwaun Generation Site',
 u'Hinkley Point C',
 u'Hornsea Phase 1',
 u'Hornsea Phase 2',
 u'Hornsea Phase 3',
 u'Hornsea Project 2 Phase 1',
 u'Hornsea Project 2 Phase 2',
 u'Hornsea Project 2 Phase 3',
 u'IPIF Fort Industrial REC',
 u'K3 CHP Facility',
 u'Kype Muir Wind Farm',
 u'Lynemouth Power Station',
 u'Middle Muir Wind Farm',
 u'Moor House Wind Farm',
 u'Moray Offshore Windfarm (East) Phase 1',
 u'Moray Offshore Windfarm (East) Phase 2',
 u'Moray Offshore Win

In [58]:
df[['Site Name', 'Installed Capacity (MWelec)',
    'Development Status (short)']][(df['Site Name'].str.contains('Blackbridge'))]

4,Site Name,Installed Capacity (MWelec),Development Status (short)
4848,Milford Haven/Blackbridge Biomass,49.9,Application Refused
5059,Blackbridge TGS 1,5.6,Abandoned


In [ ]:
# append under construction sites to the existing sites



In [18]:
building

,Site,Capacity (MW),Region,Latitude,Longitude,CHP,Company,Fuel,Type,yearStart,yearEnd
92,Port Clarence Biomass Plant,40.000,North East,54.587779,-1.230733,Yes,Glennmont Partners,Biomass,NaN,2019,2019
95,Merevale & Blyth Estate (Biomass Plant),2.500,West Midlands,52.571658,-1.586704,Yes,Merevale and Blyth Estates,Biomass,NaN,2019,2019
101,Enviroparks Hirwaun Generation Site,20.000,Wales,51.750094,-3.538987,No,Enviroparks Operations,Waste,Advanced Conversion,2019,2019
133,Tees Renewable Energy Plant,299.000,North East,54.602146,-1.161282,Yes,Macquarie/ PKA/ MGT Power,Biomass,NaN,2019,2019
141,Brite Partnership Biomass Plant / Rotherham,41.000,Yorkshire and Humber,53.422585,-1.373370,Yes,Brite Partnership Copenhagen Infrastructure Pa...,Biomass,NaN,2019,2019
144,"Pulse Park, Wellingborough",5.000,East Midlands,52.334377,-0.695258,Yes,Bester Generacion,Biomass,NaN,2019,2019
150,Charlesfield Biomass CHP Plant,10.000,Scotland,55.560490,-2.653289,Yes,Charlesfield First LLP & Biogas Power,Waste,Advanced Conversion,2019,2019
152,Discovery Park,16.000,South East,51.283909,1.345666,Yes,Burmeister & Wain Scandinavian Contractor (BWSC),Biomass,NaN,2019,2019
261,South Kirkby Business Park (Waste AD),6.000,Yorkshire and Humber,53.602657,-1.340697,No,Shanks Group,Waste,Anaerobic Digestion,2019,2019
272,Queen Charlton Quarry Renewable Energy Facilit...,2.200,South West,51.390663,-2.527469,No,Resourceful Earth,Waste,Anaerobic Digestion,2019,2019


In [129]:
df[[
    'Site Name', 'Installed Capacity (MWelec)', 'Technology Type',
    'Planning Permission Granted', 'Planning Permission Expired',
    'Development Status'
]][df['Development Status (short)'] == 'Awaiting Construction'].sort_values(
    by='Installed Capacity (MWelec)', ascending=False)

4,Site Name,Installed Capacity (MWelec),Technology Type,Planning Permission Granted,Planning Permission Expired,Development Status
2069,Dogger Bank Creyke Beck A & B,2400.0,Wind Offshore,2015-02-17 00:00:00,2020-03-11 00:00:00,Planning Permission Granted
2034,Hornsea Project Two - Optimus and Breesea,1400.0,Wind Offshore,2016-08-16 00:00:00,NaN,Planning Permission Granted
2011,East Anglia 3,1200.0,Wind Offshore,2017-08-07 00:00:00,NaN,Planning Permission Granted
2048,Dogger Bank Teesside A,1200.0,Wind Offshore,2015-08-05 00:00:00,NaN,Planning Permission Granted
5095,Sofia (Dogger Bank Teesside B),1200.0,Wind Offshore,2015-08-05 00:00:00,NaN,Planning Permission Granted
2060,Triton Knoll,860.0,Wind Offshore,2013-07-11 00:00:00,NaN,Planning Permission Granted
2057,Inch Cape,784.0,Wind Offshore,2017-05-16 00:00:00,NaN,Planning Permission Granted
5225,Coire Glas - Phase I,600.0,Pumped Storage Hydroelectricity,2013-12-13 00:00:00,NaN,Planning Permission Granted
2068,Seagreen Bravo,525.0,Wind Offshore,2017-05-16 00:00:00,NaN,Planning Permission Granted
2059,Seagreen Alpha,525.0,Wind Offshore,2017-05-16 00:00:00,NaN,Planning Permission Granted


In [132]:
df[[
    'Site Name', 'Installed Capacity (MWelec)', 'Technology Type',
    'Planning Permission Granted', 'Planning Permission Expired',
    'Development Status'
]][df['Development Status (short)'] == 'Awaiting Construction'].pivot_table(index='Technology Type',values='Installed Capacity (MWelec)',aggfunc='sum').sort_values(by='Installed Capacity (MWelec)',ascending=False)

4,Installed Capacity (MWelec)
Technology Type,
Wind Offshore,11516.000
Wind Onshore,4040.300
Battery,2671.039
Solar Photovoltaics,1710.278
Pumped Storage Hydroelectricity,1149.800
Biomass (dedicated),771.400
EfW Incineration,633.350
Advanced Conversion Technologies,598.990
Tidal Barrage and Tidal Stream,484.700


In [119]:
df['CfD Capacity (MW)'].replace('n/a',np.nan,inplace=True)
df['CfD Capacity (MW)'][(df['Development Status']=='Planning Permission Granted')].sum()

523.75

4223.426

In [101]:
df.columns

Index([                             u'Old Ref ID',
                                        u'Ref ID',
              u'Record Last Updated (dd/mm/yyyy)',
                       u'Operator (or Applicant)',
                                     u'Site Name',
                               u'Technology Type',
                                  u'Storage Type',
               u'Storage Co-location REPD Ref ID',
                   u'Installed Capacity (MWelec)',
                                   u'CHP Enabled',
                          u'RO Banding (ROC/MWh)',
                            u'FiT Tariff (p/kWh)',
                             u'CfD Capacity (MW)',
                         u'Turbine Capacity (MW)',
                               u'No. of Turbines',
                        u'Height of Turbines (m)',
                       u'Mounting Type for Solar',
                            u'Development Status',
                    u'Development Status (short)',
                               

In [95]:
# 10 largest renewable power plants

operational.sort_values(by='Capacity (MW)',ascending=False).head(10)

,Site,Capacity (MW),Region,Latitude,Longitude,Year Open,CHP,Company,Fuel,Type,yearStart,yearEnd
5129,Drax Biomass,2580.0,Yorkshire and Humber,53.735589,-0.991097,2018,n/a,Drax Power,Biomass,NaN,2018,2018
154,Drax Biomass,1935.0,Yorkshire and Humber,53.735589,-0.991097,2017,No,Drax Power,Biomass,NaN,2017,2017
5195,Dinorwig,1728.0,Wales,53.118607,-4.113893,1983,n/a,First Hydro Company,Storage,Pumped Hydro,2004,2018
170,Drax Biomass,1290.0,Yorkshire and Humber,53.735589,-0.991097,2014,No,Drax Power,Biomass,NaN,2014,2016
2061,Walney 3,660.0,Offshore,54.089995,-3.740011,2018,n/a,Orsted (formerly Dong Energy),Wind,Offshore Wind,2018,2018
153,Drax Biomass,645.0,Yorkshire and Humber,53.735589,-0.991097,2013,No,Drax Power,Biomass,Co-firing,2013,2013
2042,London Array Phase 1,630.0,Offshore,51.621679,1.495832,2013,n/a,Siemens Gamesa Renewable Energy,Wind,Offshore Wind,2013,2018
2070,Gwynt y Mor,576.0,Offshore,53.454003,-3.626649,2015,n/a,RWE,Wind,Offshore Wind,2015,2018
2041,Greater Gabbard Wind Farm,504.0,Offshore,51.917645,1.928420,2012,n/a,Airtricity / Fluor Ltd,Wind,Offshore Wind,2012,2018
5197,Cruachan,440.0,Scotland,56.411152,-5.113451,1966,n/a,Scottish Power,Storage,Pumped Hydro,2004,2018


In [ ]:
# change names to match the conventional plants data
# enter yearStart and yearEnd
# do something for contracted not built capacity / under construction stuff

In [60]:
operational['Technology Type'].unique()

array([u'Biomass (dedicated)', u'Advanced Conversion Technologies',
       u'Anaerobic Digestion', u'EfW Incineration', u'Biomass (co-firing)',
       u'Large Hydro', u'Small Hydro', u'Landfill Gas',
       u'Solar Photovoltaics', u'Sewage Sludge Digestion',
       u'Tidal Barrage and Tidal Stream', u'Shoreline Wave',
       u'Wind Offshore', u'Wind Onshore',
       u'Pumped Storage Hydroelectricity', u'Battery', u'Flywheels'], dtype=object)

In [58]:
df.columns

Index([                             u'Old Ref ID',
                                        u'Ref ID',
              u'Record Last Updated (dd/mm/yyyy)',
                       u'Operator (or Applicant)',
                                     u'Site Name',
                               u'Technology Type',
                                  u'Storage Type',
               u'Storage Co-location REPD Ref ID',
                   u'Installed Capacity (MWelec)',
                                   u'CHP Enabled',
                          u'RO Banding (ROC/MWh)',
                            u'FiT Tariff (p/kWh)',
                             u'CfD Capacity (MW)',
                         u'Turbine Capacity (MW)',
                               u'No. of Turbines',
                        u'Height of Turbines (m)',
                       u'Mounting Type for Solar',
                            u'Development Status',
                    u'Development Status (short)',
                               

In [57]:
#operational['Development Status'].unique()
operational.head()

4,Site Name,Technology Type,Installed Capacity (MWelec),Region,lat,lon,Year Open
18,Chilton Energy Plant,Biomass (dedicated),18.0,North East,54.668027,-1.566632,2012
21,Double H Nurseries Biomass Plant,Biomass (dedicated),1.5,South East,50.750782,-1.678119,2012
22,Newry Biomass Phase 1 (Gasification),Advanced Conversion Technologies,2.0,Northern Ireland,54.217507,-6.336669,2012
23,Beckton STW Biodiesel Power Plant,Biomass (dedicated),8.0,London,51.517777,0.093461,2010
24,Buslingthorpe Power Station (Leeds North),Biomass (dedicated),2.0,Yorkshire and Humber,53.814327,-1.542615,2011


In [80]:
#df['X-coordinate'] = pd.to_numeric(df['X-coordinate'],errors='coerce')
df["X-coordinate"] = df["X-coordinate"].astype(float)

In [107]:
#df["X-coordinate"].loc[df["X-coordinate"] == NaN]
df[['X-coordinate','Y-coordinate','Installed Capacity (MWelec)','Development Status','Site Name']].loc[df['X-coordinate'].isnull()]
df[['Installed Capacity (MWelec)']].loc[(df['X-coordinate'].isnull())&(df['Development Status']=='Operational')].sum()

4
Installed Capacity (MWelec)    0.0
dtype: float64

In [116]:
df[['X-coordinate','Y-coordinate','lat','lon']].loc[[1200,1201,1202,5179]]

4,X-coordinate,Y-coordinate,lat,lon
1200,464288.0,405319.0,53.540590,-1.031381
1201,-477441.0,481803.0,53.492552,-15.262149
1202,538495.0,348945.0,53.019888,0.063251
5179,147788.0,570314.0,54.963451,-5.941645


In [72]:
df[['Development Status','Site Name','Installed Capacity (MWelec)']].loc[[5176,5177,5178,5179]]

4,Development Status,Site Name,Installed Capacity (MWelec)
5176,Planning Application Submitted,Aviva Headquarters Perth,0.85
5177,Planning Application Submitted,Whapps Town Windfarm (Resubmission),9.2
5178,Appeal Lodged,Lands 260m SW of 76 Strabane Road,0.25
5179,Appeal Lodged,402 metres NE of 45 Dickeystown Road,0.25


In [24]:
df.columns.tolist()

[u'Old Ref ID',
 u'Ref ID',
 u'Record Last Updated (dd/mm/yyyy)',
 u'Operator (or Applicant)',
 u'Site Name',
 u'Technology Type',
 u'Storage Type',
 u'Storage Co-location REPD Ref ID',
 u'Installed Capacity (MWelec)',
 u'CHP Enabled',
 u'RO Banding (ROC/MWh)',
 u'FiT Tariff (p/kWh)',
 u'CfD Capacity (MW)',
 u'Turbine Capacity (MW)',
 u'No. of Turbines',
 u'Height of Turbines (m)',
 u'Mounting Type for Solar',
 u'Development Status',
 u'Development Status (short)',
 u'Address',
 u'County',
 u'Region',
 u'Country',
 u'Post Code',
 u'X-coordinate',
 u'Y-coordinate',
 u'Planning Authority',
 u'Planning Application Reference',
 u'Appeal Reference',
 u'Secretary of State Reference',
 u'Type of Secretary of State Intervention',
 u'Judicial Review',
 u'Offshore Wind Round',
 u'Planning Application Submitted',
 u'Planning Application Withdrawn',
 u'Planning Permission Refused',
 u'Appeal Lodged',
 u'Appeal Withdrawn',
 u'Appeal Refused',
 u'Appeal Granted',
 u'Planning Permission Granted',
 u'

In [6]:
# df.dtypes
print df['Record Last Updated (dd/mm/yyyy)'].tail(20)

5432    2018-11-27 00:00:00
5433    2018-11-27 00:00:00
5434    2018-11-28 00:00:00
5435    2018-11-28 00:00:00
5436    2018-11-29 00:00:00
5437    2019-01-02 00:00:00
5438    2018-12-04 00:00:00
5439    2018-12-06 00:00:00
5440    2018-12-06 00:00:00
5441    2018-12-06 00:00:00
5442    2018-12-06 00:00:00
5443    2018-12-06 00:00:00
5444    2018-12-06 00:00:00
5445    2018-12-06 00:00:00
5446    2018-12-06 00:00:00
5447    2018-12-06 00:00:00
5448    2018-12-20 00:00:00
5449    2018-12-20 00:00:00
5450    2018-12-20 00:00:00
5451    2019-01-03 00:00:00
Name: Record Last Updated (dd/mm/yyyy), dtype: object


In [7]:
# drop non-data rows and columns

df.drop(df.index[:5], inplace=True)
df.columns.values[0] = 'blank1'
df.columns.values[1] = 'blank2'
# drop first two columns that are blank
df.drop(['blank1','blank2'], axis=1, inplace=True)

#df.head()

4,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,...,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational
10,70700000,6,2004-08-13 00:00:00,Incetec / Border Biofuels,Kingmoor Marshalling Yard,Biomass (dedicated),NaN,NaN,20,No,...,NaN,NaN,NaN,2000-03-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
11,9162000,7,2004-11-29 00:00:00,NOVERA,North Wiltshire Biomass Power Plant,Biomass (dedicated),NaN,NaN,5.528,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,100640000,8,2003-12-23 00:00:00,Private Developer,Newbridge Power Station,Biomass (dedicated),NaN,NaN,15,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,11065000,9,2004-08-30 00:00:00,Agricultural Energy Company Limited,Poole Biomass Plant,Biomass (dedicated),NaN,NaN,10.93,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,12076000,10,2012-01-04 00:00:00,Yorkshire Environmental,Eggborough Biomass Power Station,Advanced Conversion Technologies,NaN,NaN,10.62,No,...,NaN,NaN,NaN,1997-01-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#print df['X-coordinate'].loc[df['X-coordinate'].isna()]
#print df.loc[1301]
df['X-coordinate'].head()

10    337200
11    411500
12    302400
13    314900
14    457200
Name: X-coordinate, dtype: object

In [56]:
# print df.loc[np.isnan(df['X-coordinate']), 'Installed Capacity (MWelec)']

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [6]:
# fill empty x and y coords with locations as follows:
df['X-coordinate'] = df['X-coordinate'].fillna(-477441)
df['Y-coordinate'] = df['Y-coordinate'].fillna(481803)
print df[['X-coordinate','Y-coordinate']].loc[[1301,1302]]

4    X-coordinate  Y-coordinate
1301       359296      288893.0
1302       182941       59990.0


In [23]:
#df[['Y-coordinate']].isnull()*53.492548

In [81]:
def proj_transform(df):
    bng = pyproj.Proj(init='epsg:27700')
    wgs84 = pyproj.Proj(init='epsg:4326')
    lon,lat = pyproj.transform(bng,wgs84, df['X-coordinate'], df['Y-coordinate'])    
    df['lat'] = lat
    df['lon'] = lon
    return df

df = df.apply(proj_transform, axis=1)
print df[['X-coordinate', 'Y-coordinate','lat','lon']].loc[[1301,1302]]

RuntimeError: ('non-convergent inverse meridional dist', u'occurred at index 1201')

In [25]:
summary = df.groupby(['Technology Type','Development Status']).sum().reset_index()
summary[['Technology Type','Development Status', 'Installed Capacity (MWelec)']].to_csv('summary.csv')
print summary[['Technology Type','Development Status', 'Installed Capacity (MWelec)']].tail(10)

4   Technology Type               Development Status  \
118    Wind Onshore                   Decommissioned   
119    Wind Onshore                      Operational   
120    Wind Onshore   Planning Application Submitted   
121    Wind Onshore   Planning Application Withdrawn   
122    Wind Onshore      Planning Permission Expired   
123    Wind Onshore      Planning Permission Granted   
124    Wind Onshore      Planning Permission Refused   
125    Wind Onshore  Secretary of State - Intervened   
126    Wind Onshore     Secretary of State - Refusal   
127    Wind Onshore               Under Construction   

4    Installed Capacity (MWelec)  
118                      38.0500  
119                   12151.4920  
120                    2180.3500  
121                    6269.4018  
122                      73.4000  
123                    2467.5500  
124                    6266.0114  
125                      40.0400  
126                     189.4000  
127                    1217.7500 

In [26]:
pivot = df.pivot_table(index='Development Status', columns='Technology Type', values='Installed Capacity (MWelec)', aggfunc='sum')
print pivot

Technology Type                  Advanced Conversion Technologies  \
Development Status                                                  
Abandoned                                                  294.62   
Appeal Granted                                              11.00   
Appeal Lodged                                               12.00   
Appeal Refused                                              16.00   
Appeal Withdrawn                                            31.00   
Decommissioned                                               9.20   
No Application Required                                       NaN   
Operational                                                 70.30   
Planning Application Submitted                              81.00   
Planning Application Withdrawn                             242.80   
Planning Permission Expired                                  9.00   
Planning Permission Granted                                645.99   
Planning Permission Refused       

In [40]:
# how many onshore windfarms submitted planning applications after July 2015?
# select onshore windfarms
onwind = df.loc[df['Technology Type'] == 'Wind Onshore']
# select apps submitted after 2015
recent_onwind = onwind.loc[onwind['Planning Application Submitted'].dt.year >= 2015]
recent_onwind = recent_onwind.loc[(recent_onwind['Planning Application Submitted'].dt.month > 7) |
                                   (recent_onwind['Planning Application Submitted'].dt.year > 2015)]
print len(recent_onwind)
print recent_onwind

# get columns headers
# onwind.columns.tolist()

# check how many null values
# print onwind['Planning Application Submitted'].isnull().sum()
# check how many rows
# print len(onwind)


123
4    Old Ref ID Ref ID Record Last Updated (dd/mm/yyyy)  \
2214      AA184   2767              2018-08-20 00:00:00   
2250      B0705   2847              2018-02-28 00:00:00   
2251      B0710   2848              2017-04-04 00:00:00   
2255      B0782   2860              2017-07-28 00:00:00   
2653      C2732   3373              2018-08-03 00:00:00   
2662      B0801   3387              2018-04-03 00:00:00   
2663      C0716   3388              2018-04-30 00:00:00   
2695      C1798   3430              2018-06-20 00:00:00   
2696      C2520   3431              2016-12-01 00:00:00   
2753      AA995   3502              2017-11-30 00:00:00   
2767      B0337   3516              2018-09-03 00:00:00   
2856      B1321   3629              2018-06-22 00:00:00   
3200     IF1375   4003              2018-07-23 00:00:00   
3515      C1030   4334              2016-02-05 00:00:00   
3595      B1493   4420              2018-09-04 00:00:00   
3649      C1581   4487              2017-05-17 00:00

In [5]:
df['Year opened'] = df['Operational'].dt.year
print df['Year opened'].head(10)
# only keep records that are operational
operating = df.dropna(subset=['Year opened'])
print operating

10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17    2012.0
18       NaN
19       NaN
Name: Year opened, dtype: float64
4    Old Ref ID Ref ID Record Last Updated (dd/mm/yyyy)  \
17        A0331     14              2012-04-04 00:00:00   
20        AA851     22              2012-08-10 00:00:00   
21        B0242     23              2016-10-25 00:00:00   
22        B0376     24              2012-05-29 00:00:00   
23        B0461     26              2012-08-07 00:00:00   
25      EN00001     28              2010-06-09 00:00:00   
27      EN00121     33              2013-10-08 00:00:00   
28      EN00151     34              2011-06-29 00:00:00   
29      EN00156     35              2016-05-10 00:00:00   
30      EN00236     37              2009-12-04 00:00:00   
33      EN00569     41              2010-06-09 00:00:00   
36      ENA0003     44              2006-05-23 00:00:00   
37        H0064     46              2016-09-02 00:00:00   
38       I

In [6]:
operating['Installed Capacity (MWelec)'].max()

740

In [11]:
wind = operating.loc[(operating['Technology Type']=='Wind Offshore')|(operating['Technology Type']=='Wind Onshore')]
wind[['Installed Capacity (MWelec)','Technology Type']].groupby('Technology Type').sum()

4,Installed Capacity (MWelec)
Technology Type,
Wind Offshore,7908.700
Wind Onshore,12454.742


In [84]:
# list unique types
df.columns.tolist()
# add summary type column and call onwind "wind" (need to iterate this through all types)

#print df[['Technology Type', 'summary type']]

[u'Old Ref ID',
 u'Ref ID',
 u'Record Last Updated (dd/mm/yyyy)',
 u'Operator (or Applicant)',
 u'Site Name',
 u'Technology Type',
 u'Installed Capacity (MWelec)',
 u'CHP Enabled',
 u'RO Banding (ROC/MWh)',
 u'FiT Tariff (p/kWh)',
 u'CfD Capacity (MW)',
 u'Turbine Capacity (MW)',
 u'No. of Turbines',
 u'Height of Turbines (m)',
 u'Mounting Type for Solar',
 u'Development Status',
 u'Development Status (short)',
 u'Address',
 u'County',
 u'Region',
 u'Country',
 u'Post Code',
 u'X-coordinate',
 u'Y-coordinate',
 u'Planning Authority',
 u'Planning Application Reference',
 u'Appeal Reference',
 u'Secretary of State Reference',
 u'Type of Secretary of State Intervention',
 u'Judicial Review',
 u'Offshore Wind Round',
 u'Planning Application Submitted',
 u'Planning Application Withdrawn',
 u'Planning Permission Refused',
 u'Appeal Lodged',
 u'Appeal Withdrawn',
 u'Appeal Refused',
 u'Appeal Granted',
 u'Planning Permission Granted',
 u'Secretary of State - Intervened',
 u'Secretary of State

In [6]:
df.head(10)

,Date,Nuclear,Biomass,Imports,Hydro,Wind,Solar,Gas,Coal,Demand,year,month,day,total,RE,lowc,fossil,coalzerohh
0,2009-01-01,6973.0,0.0,1412.0,246.0,148.0,0,11900.0,17650.0,38329.0,2009,1,1,36917.0,394.0,7367.0,29550.0,False
1,2009-01-01,6968.0,0.0,1290.0,245.0,157.0,0,12031.0,17770.0,38461.0,2009,1,1,37171.0,402.0,7370.0,29801.0,False
2,2009-01-01,6970.0,0.0,799.0,246.0,147.0,0,11754.0,18070.0,37986.0,2009,1,1,37187.0,393.0,7363.0,29824.0,False
3,2009-01-01,6969.0,0.0,317.0,246.0,148.0,0,11162.0,18022.0,36864.0,2009,1,1,36547.0,394.0,7363.0,29184.0,False
4,2009-01-01,6960.0,0.0,4.0,246.0,160.0,0,10812.0,17998.0,36180.0,2009,1,1,36176.0,406.0,7366.0,28810.0,False
5,2009-01-01,6969.0,0.0,-116.0,236.0,151.0,0,10698.0,17935.0,35873.0,2009,1,1,35989.0,387.0,7356.0,28633.0,False
6,2009-01-01,6964.0,0.0,-907.0,226.0,124.0,0,10735.0,17564.0,34706.0,2009,1,1,35613.0,350.0,7314.0,28299.0,False
7,2009-01-01,6967.0,0.0,-1020.0,225.0,161.0,0,10481.0,16827.0,33641.0,2009,1,1,34661.0,386.0,7353.0,27308.0,False
8,2009-01-01,6963.0,0.0,-1114.0,225.0,154.0,0,9841.0,16453.0,32522.0,2009,1,1,33636.0,379.0,7342.0,26294.0,False
9,2009-01-01,6960.0,0.0,-1088.0,224.0,165.0,0,9556.0,16037.0,31854.0,2009,1,1,32942.0,389.0,7349.0,25593.0,False


In [7]:
df.set_index('Date', inplace=True)

In [8]:
results = df['Nuclear'].head(0)

df.sort_values(['Nuclear'], ascending=False, inplace=True)
results['Nuclear'] = df['Nuclear'].head(10)

df.sort_values(['Biomass'], ascending=False, inplace=True)
results['Biomass'] = df['Biomass'].head(10)

df.sort_values(['Hydro'], ascending=False, inplace=True)
results['Hydro'] = df['Hydro'].head(10)

df.sort_values(['Wind'], ascending=False, inplace=True)
results['Wind'] = df['Wind'].head(10)

df.sort_values(['Solar'], ascending=False, inplace=True)
results['Solar'] = df['Solar'].head(10)

df.sort_values(['Gas'], ascending=False, inplace=True)
results['Gas'] = df['Gas'].head(10)

df.sort_values(['Coal'], ascending=False, inplace=True)
results['Coal'] = df['Coal'].head(10)

results.to_csv('10_largest_vals_with_dates.csv')

In [9]:
df.describe()

,Nuclear,Biomass,Imports,Hydro,Wind,Solar,Gas,Coal,Demand,year,month,day,total,RE,lowc,fossil
count,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000,152925.000000
mean,7335.656204,653.908021,1205.469247,388.239655,2566.829308,474.535132,12777.886805,10210.493116,35626.276912,2012.873742,6.387778,15.696322,34407.548241,4083.512116,11419.168320,22988.379921
std,961.759615,731.201295,1636.807815,249.512969,2056.138355,1106.769477,5203.749404,6096.520493,7758.558011,2.520646,3.411183,8.792397,7659.202836,3022.037935,3253.456165,8333.597104
min,4099.000000,0.000000,-5305.000000,0.000000,8.000000,0.000000,1556.000000,0.000000,18320.000000,2009.000000,1.000000,1.000000,18090.000000,57.000000,4703.000000,3676.000000
25%,6760.000000,0.000000,144.000000,181.000000,969.000000,0.000000,8751.000000,5139.000000,29686.000000,2011.000000,3.000000,8.000000,28522.000000,1640.000000,9052.000000,16707.000000
50%,7475.000000,474.000000,1473.000000,351.000000,1952.000000,0.000000,13072.000000,10358.000000,35864.000000,2013.000000,6.000000,16.000000,34255.000000,3299.000000,10746.000000,22498.000000
75%,8047.000000,1119.000000,2512.000000,558.000000,3680.000000,261.000000,16863.000000,14798.000000,40948.000000,2015.000000,9.000000,23.000000,39686.000000,5910.000000,13353.000000,28986.000000
max,9342.000000,2441.000000,5412.000000,1403.000000,10823.000000,8910.000000,26566.000000,26044.000000,60070.000000,2017.000000,12.000000,31.000000,58578.000000,19154.000000,27318.000000,48963.000000


In [10]:
print df['Nuclear'].max()
print df['Nuclear'].idxmax()

9342.0
2013-08-19 00:00:00


In [11]:
print df.groupby(['year','month']).mean().reset_index()

     year  month      Nuclear      Biomass      Imports       Hydro  \
0    2009      1  5794.037106     0.000000  -187.636728  545.584406   
1    2009      2  8063.098958     0.000000   -55.710565  403.909970   
2    2009      3  8522.880777     0.000000    84.449151  579.504571   
3    2009      4  8389.706796     0.000000   896.759799  388.635736   
4    2009      5  6826.467742     0.000000  1351.060484  308.965726   
5    2009      6  8004.785932     0.000000   643.291728  153.376877   
6    2009      7  7874.081592     0.000000   645.163019  167.533570   
7    2009      8  7686.621965     0.000000   188.858265  370.462405   
8    2009      9  7023.880053     0.000000  -641.437564  483.439518   
9    2009     10  6353.609825     0.000000 -1201.963658  437.858416   
10   2009     11  7277.863940     0.000000  -799.442552  608.236202   
11   2009     12  7323.837753     0.000000  -801.124074  430.944863   
12   2010      1  8015.969434     0.000000 -1361.371832  258.286050   
13   2

In [39]:
monthly = df.groupby(['year','month']).mean().reset_index()
monthly['windcoal'] = monthly['Wind'] > monthly['Coal']
monthly['solarcoal'] = monthly['Solar'] > monthly['Coal']
monthly['nuc%'] = monthly['Nuclear']/monthly['total']
monthly['bio%'] = monthly['Biomass']/monthly['total']
monthly['hyd%'] = monthly['Hydro']/monthly['total']
monthly['win%'] = monthly['Wind']/monthly['total']
monthly['sol%'] = monthly['Solar']/monthly['total']
monthly['gas%'] = monthly['Gas']/monthly['total']
monthly['coa%'] = monthly['Coal']/monthly['total']
monthly['RE%'] = monthly['RE']/monthly['total']
monthly['lowc%'] = monthly['lowc']/monthly['total']


In [40]:
monthly.to_csv('power_by_month.csv')

In [38]:
daily = df.groupby(['year','month','day']).mean().reset_index()
daily['windcoal'] = daily['Wind'] > daily['Coal']
daily['solarcoal'] = daily['Solar'] > daily['Coal']
daily['coalzeroday'] = daily['Coal'] < 0.01
daily['nuc%'] = daily['Nuclear']/daily['total']
daily['bio%'] = daily['Biomass']/daily['total']
daily['hyd%'] = daily['Hydro']/daily['total']
daily['win%'] = daily['Wind']/daily['total']
daily['sol%'] = daily['Solar']/daily['total']
daily['gas%'] = daily['Gas']/daily['total']
daily['coa%'] = daily['Coal']/daily['total']
daily['RE%'] = daily['RE']/daily['total']
daily['lowc%'] = daily['lowc']/daily['total']
daily.to_csv('power_by_day.csv')

In [28]:
dailycounts = daily.groupby(['year']).sum().reset_index()
dailycounts[['year','windcoal','solarcoal','coalzeroday']].to_csv('dailycounts.csv')

In [16]:
temp01 = df.groupby(['year','month']).sum().reset_index()
temp02 = temp01[['year','month','coalzerohh']]
temp02.to_csv('coal_free_HH.csv')

In [37]:
daily.head(10)

,year,month,day,Nuclear,Biomass,Imports,Hydro,Wind,Solar,Gas,...,Demand,total,RE,lowc,fossil,coalzerohh,windcoal,solarcoal,coalzeroday,nuc%
0,2009,1,1,6967.541667,0.0,1166.562500,309.166667,152.062500,0.0,11578.791667,...,37845.145833,36678.583333,461.229167,7428.770833,29249.812500,0.0,False,False,False,0.189962
1,2009,1,2,6398.958333,0.0,209.208333,322.270833,360.520833,0.0,14397.083333,...,40837.187500,40627.979167,682.791667,7081.750000,33546.229167,0.0,False,False,False,0.157501
2,2009,1,3,5798.312500,0.0,349.750000,356.145833,339.666667,0.0,15162.770833,...,40246.875000,39890.041667,695.812500,6494.125000,33395.916667,0.0,False,False,False,0.145357
3,2009,1,4,5468.229167,0.0,85.812500,339.270833,458.312500,0.0,16307.125000,...,40716.062500,40529.083333,797.583333,6265.812500,34263.270833,0.0,False,False,False,0.134921
4,2009,1,5,5130.208333,0.0,-636.729167,447.000000,645.312500,0.0,18964.979167,...,46577.208333,46599.458333,1092.312500,6222.520833,40376.937500,0.0,False,False,False,0.110092
5,2009,1,6,5134.083333,0.0,-1040.895833,358.833333,541.229167,0.0,19026.270833,...,48078.041667,48267.604167,900.062500,6034.145833,42233.458333,0.0,False,False,False,0.106367
6,2009,1,7,5482.416667,0.0,-458.979167,358.541667,233.000000,0.0,18628.666667,...,48130.270833,48000.958333,591.541667,6073.958333,41927.000000,0.0,False,False,False,0.114215
7,2009,1,8,5520.729167,0.0,-721.791667,337.000000,562.645833,0.0,17661.687500,...,46920.562500,47107.645833,899.645833,6420.375000,40687.270833,0.0,False,False,False,0.117194
8,2009,1,9,5230.625000,0.0,-1370.479167,389.875000,726.770833,0.0,17726.250000,...,46754.416667,47355.500000,1116.645833,6347.270833,41008.229167,0.0,False,False,False,0.110454
9,2009,1,10,5584.791667,0.0,-481.333333,431.750000,1764.833333,0.0,15408.666667,...,43283.791667,43631.166667,2196.583333,7781.375000,35849.791667,0.0,False,False,False,0.128000


In [47]:
temp01 = daily.groupby(['year']).count().reset_index()
temp02 = temp01[['year','day']]
temp02.head(10)

,year,day
0,2009,365
1,2010,365
2,2011,365
3,2012,366
4,2013,365
5,2014,365
6,2015,365
7,2016,366
8,2017,264


In [42]:
monthlycounts = monthly.groupby(['year']).sum().reset_index()
monthlycounts[['year','windcoal','solarcoal']].to_csv('monthlycounts.csv')

In [22]:
onwind = df.loc[(df['Technology Type']=='Wind Onshore') & (df['Development Status']=='Operational')]
onwind['year'] = onwind['Operational'].dt.year
onwind['month'] = onwind['Operational'].dt.month
onwind[['Installed Capacity (MWelec)','year','month']].groupby(['year','month']).sum().reset_index().to_csv('test.csv')

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
onapp = df.loc[(df['Technology Type']=='Wind Onshore')]
onapp['year'] = onapp['Planning Application Submitted'].dt.year
onapp['month'] = onapp['Planning Application Submitted'].dt.month
onapp[['Installed Capacity (MWelec)','year','month']].groupby(['year','month']).sum().reset_index().to_csv('onapp.csv')

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
onconstr = df.loc[(df['Technology Type']=='Wind Onshore')&(df['Development Status']=='Under Construction')]
onconstr['year'] = onconstr['Planning Application Submitted'].dt.year
onapp['month'] = onapp['Planning Application Submitted'].dt.month
onapp[['Installed Capacity (MWelec)','year','month']].groupby(['year','month']).sum().reset_index().to_csv('onapp.csv')

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
constr = df[['Country','Installed Capacity (MWelec)']].loc[(df['Technology Type']=='Wind Onshore')&(df['Development Status']=='Under Construction')].groupby('Country').sum().reset_index()
constr.columns = ['Country','Under construction']
print constr['Under construction'].sum()
print constr

1253.0
            Country  Under construction
0           England               34.65
1  Northern Ireland              151.30
2          Scotland              825.45
3             Wales              241.60


In [65]:
df['Country'].replace('Northern Ireland ','Northern Ireland',inplace=True)
df['Country'] = df['Country'].str.strip()
onperm = df[['Country','Installed Capacity (MWelec)']].loc[(df['Development Status'].str.contains('Granted'))&(df['Technology Type']=='Wind Onshore')].groupby('Country').sum().reset_index()
onperm.columns = ['Country','Planning permission']
print onperm['Planning permission'].sum()
print onperm

3811.15
            Country  Planning permission
0           England               112.25
1  Northern Ireland               253.80
2          Scotland              3167.30
3             Wales               277.80


In [72]:
pd.options.display.float_format = '{:,.0f}'.format
table = pd.merge(constr,onperm,left_on='Country',right_on='Country')
table.loc['total'] = table.sum()
table['Country'].replace('EnglandNorthern IrelandScotlandWales','Total',inplace=True)
table

,Country,Under construction,Planning permission
0,England,35,112
1,Northern Ireland,151,254
2,Scotland,825,"3,167"
3,Wales,242,278
total,Total,"1,253","3,811"


In [48]:
df[['Country','Installed Capacity (MWelec)']].loc[(df['Development Status'].str.contains('Granted'))&(df['Technology Type']=='Wind Onshore')].groupby('Country').sum().reset_index()

4,Country,Installed Capacity (MWelec)
0,England,112.25
1,Northern Ireland,244.80
2,Northern Ireland,9.00
3,Scotland,3167.30
4,Wales,277.80


In [53]:
df['Country'].unique()

array([u'Wales', u'Scotland', u'England', u'Northern Ireland', nan,
       u'England ', u'Northern Ireland '], dtype=object)

In [11]:
onconstr[['Installed Capacity (MWelec)','year']].groupby(['year']).sum().reset_index()

4,year,Installed Capacity (MWelec)
0,2003,4.0
1,2004,50.0
2,2005,53.0
3,2006,88.7
4,2007,21.0
5,2008,200.0
6,2009,15.4
7,2010,4.5
8,2011,214.8
9,2012,374.5


In [41]:
onwind
onwind['app year'] = onwind['Planning Application Submitted'].dt.year
onwind['app month'] = onwind['Planning Application Submitted'].dt.month


onwind[['Installed Capacity (MWelec)','Country','Planning Application Submitted','app year','app month']].groupby(['Country','app year','app month']).agg({'Installed Capacity (MWelec)':'sum','Planning Application Submitted':'count'}).reset_index().to_csv('windapps.csv', index=False)


onwind['grant year'] = onwind['Planning Permission Granted'].dt.year
onwind['grant month'] = onwind['Planning Permission Granted'].dt.month


onwind[['Installed Capacity (MWelec)','Country','Planning Permission Granted','grant year','grant month']].groupby(['Country','grant year','grant month']).agg({'Installed Capacity (MWelec)':'sum','Planning Permission Granted':'count'}).reset_index().to_csv('windgranteds.csv', index=False)
onwind['Development Status (short)'].unique()

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

array([u'Application Refused', u'Abandoned', u'Operational',
       u'Application Withdrawn', u'Awaiting Construction',
       u'Under Construction', u'Decommissioned',
       u'Planning Permission Expired', u'Application Submitted'], dtype=object)

In [25]:
england = onwind.loc[(onwind['Planning Permission Granted'].dt.year >= 2015)&(onwind['Planning Permission Granted'].dt.month >= 5)&(onwind['Country'] == 'England')]
england.to_csv('since_reset.csv',index=False)

being_built = onwind.loc[(onwind['Development Status'] == 'Under Construction')&(onwind['Country'] == 'England')]
being_built.to_csv('being_built.csv',index=False)

In [42]:
consented = onwind.loc[(onwind['Development Status (short)']=='Awaiting Construction')]
consented['Installed Capacity (MWelec)'] = consented[['Installed Capacity (MWelec)']].apply(
    pd.to_numeric, errors='coerce')
print consented['Installed Capacity (MWelec)'].sum()
#consented.pivot_table(columns='Country',values='Installed Capacity (MWelec)',aggfunc='sum' ).reset_index()
#consented[['Country','grant year','Installed Capacity (MWelec)']].groupby(['Country','grant year']).agg({'Installed Capacity (MWelec)':['sum','mean','count']})
#table.fillna(0)

#print onwind['Installed Capacity (MWelec)'].loc[(onwind['Development Status (short)']=='Awaiting Construction')&(onwind['Country']=='Scotland')&(onwind['grant year']>=2018)].mean()
#print onwind['Installed Capacity (MWelec)'].loc[(onwind['Development Status (short)']=='Awaiting Construction')&(onwind['Country']=='England')].mean()
onwind['Development Status (short)'].unique()

3884.55


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([u'Application Refused', u'Abandoned', u'Operational',
       u'Application Withdrawn', u'Awaiting Construction',
       u'Under Construction', u'Decommissioned',
       u'Planning Permission Expired', u'Application Submitted'], dtype=object)

In [43]:
consented[['Site Name','Operator (or Applicant)','Installed Capacity (MWelec)','Development Status','Development Status (short)','Planning Permission Granted','Planning Permission Expired']].loc[(consented['Country']=='England')]


4,Site Name,Operator (or Applicant),Installed Capacity (MWelec),Development Status,Development Status (short),Planning Permission Granted,Planning Permission Expired
2865,Laurels Farm - second resubmission,Stamford Renewable Power,2.00,Appeal Granted,Awaiting Construction,2014-04-02 00:00:00,NaN
2980,Gallymoor Landfill Site,FCC Environment,1.80,Planning Permission Granted,Awaiting Construction,2012-07-10 00:00:00,NaN
3338,Bullamoor,Muirhall Energy (formerly Infinis),8.00,Planning Permission Granted,Awaiting Construction,2010-12-10 00:00:00,NaN
3340,Heckington Fens,Ecotricity,54.00,Planning Permission Granted,Awaiting Construction,2013-02-08 00:00:00,NaN
3358,Carnaby,FCC Environment,6.00,Planning Permission Granted,Awaiting Construction,2012-02-20 00:00:00,NaN
3560,Hyndburn Wind Farm Extension,Energiekontor UK,8.20,Planning Permission Granted,Awaiting Construction,2015-06-26 00:00:00,NaN
3606,Withernwick extension,Energiekontor UK,8.20,Appeal Granted,Awaiting Construction,2016-12-19 00:00:00,NaN
4162,Land near Ventonteague,Cornwall Council,1.50,Planning Permission Granted,Awaiting Construction,2015-10-26 00:00:00,2018-10-26 00:00:00
4199,Smart Systems,D2Smart Projects,1.00,Planning Permission Granted,Awaiting Construction,2015-09-28 00:00:00,2018-09-28 00:00:00
4872,Checkley Wood Farm,Checkley Wood Energy,3.00,Planning Permission Granted,Awaiting Construction,2017-09-13 00:00:00,2022-09-13 00:00:00


In [50]:
consented.dtypes

4
Old Ref ID                                  object
Ref ID                                      object
Record Last Updated (dd/mm/yyyy)            object
Operator (or Applicant)                     object
Site Name                                   object
Technology Type                             object
Installed Capacity (MWelec)                 object
CHP Enabled                                 object
RO Banding (ROC/MWh)                        object
FiT Tariff (p/kWh)                          object
CfD Capacity (MW)                           object
Turbine Capacity (MW)                       object
No. of Turbines                             object
Height of Turbines (m)                      object
Mounting Type for Solar                     object
Development Status                          object
Development Status (short)                  object
Address                                     object
County                                      object
Region                       

In [10]:
df.loc[df['Technology Type'].str.contains('Hydro')]

4,nan,nan,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Installed Capacity (MWelec),CHP Enabled,...,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational
349,NaN,NaN,N00010H,388,2009-07-17 00:00:00,Scottish and Southern Energy (SSE),Glendoe Hydro Scheme,Large Hydro,100,n/a,...,NaN,NaN,NaN,2005-07-28 00:00:00,NaN,NaN,NaN,NaN,2006-05-01 00:00:00,2008-12-01 00:00:00
350,NaN,NaN,N00013H,389,2005-05-12 00:00:00,RWE npower,Stronelarig Hydro Scheme,Large Hydro,10.7,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,NaN,NaN,N00026H,390,2006-05-12 00:00:00,Scottish and Southern Energy (SSE),Fasnakyle Hydro Extension,Large Hydro,7.5,n/a,...,NaN,NaN,NaN,2004-04-01 00:00:00,NaN,NaN,NaN,NaN,2006-05-01 00:00:00,2006-08-01 00:00:00
352,NaN,NaN,N0021H,391,2018-04-04 00:00:00,Hydro Plan (on behalf of Alcan),Kinlochleven Hydro Power Station,Large Hydro,19.5,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909-01-01 00:00:00
353,NaN,NaN,EN00014,393,2006-04-04 00:00:00,Scottish and Southern Energy (SSE),Gaur,Large Hydro,6.4,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1953-01-01 00:00:00
354,NaN,NaN,EN00016,394,2006-04-04 00:00:00,Scottish and Southern Energy (SSE),Pitlochry,Large Hydro,15,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950-01-01 00:00:00
355,NaN,NaN,EN00049,395,2006-04-03 00:00:00,Scottish and Southern Energy (SSE),Loch Gair,Large Hydro,6,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961-01-01 00:00:00
356,NaN,NaN,EN00051,396,2006-04-03 00:00:00,Scottish and Southern Energy (SSE),Sron Mor,Large Hydro,5,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1957-01-01 00:00:00
357,NaN,NaN,EN00052,397,2010-03-03 00:00:00,Scottish and Southern Energy (SSE),Striven,Large Hydro,8,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951-01-01 00:00:00
358,NaN,NaN,EN00053,398,2006-04-03 00:00:00,Scottish and Southern Energy (SSE),Aigas,Large Hydro,20,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1962-01-01 00:00:00


In [6]:
df.loc[df['Site Name'].str.contains('Lynemouth')]

4,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Installed Capacity (MWelec),CHP Enabled,RO Banding (ROC/MWh),FiT Tariff (p/kWh),...,Appeal Withdrawn,Appeal Refused,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational
95,AA430,115,2017-09-01 00:00:00,Energetický a prumyslový holding (EPH),Lynemouth Power Station,Biomass (dedicated),420,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2565,N00361W,3255,2010-09-24 00:00:00,Scottish Power Renewables,Lynemouth Windfarm,Wind Onshore,2.4,n/a,NaN,NaN,...,NaN,NaN,2008-02-20 00:00:00,2008-02-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2967,AA008,3752,2012-04-24 00:00:00,Scottish Power Renewables,Lynemouth Windfarm (Resubmission),Wind Onshore,26,n/a,NaN,NaN,...,NaN,NaN,2009-01-08 00:00:00,2009-01-08 00:00:00,NaN,NaN,NaN,NaN,2010-07-01 00:00:00,2012-04-12 00:00:00


In [13]:
df[['No. of Turbines','Installed Capacity (MWelec)','Turbine Capacity (MW)']].loc[df['Development Status']=='Operational']

4,No. of Turbines,Installed Capacity (MWelec),Turbine Capacity (MW)
17,n/a,18,n/a
20,n/a,1.5,n/a
21,n/a,2,n/a
22,n/a,8,n/a
23,n/a,2,n/a
25,n/a,5,n/a
27,n/a,2.45,n/a
28,n/a,26.043,n/a
29,n/a,35.22,n/a
30,n/a,2,n/a


In [24]:
df[['Site Name','Installed Capacity (MWelec)','Development Status','Development Status (short)','CfD Capacity (MW)']].loc[(df['Technology Type'].str.contains('Offshore'))&((df['Development Status']=='Planning Permission Granted')|(df['Development Status']=='Planning permission granted'))]

4,Site Name,Installed Capacity (MWelec),Development Status,Development Status (short),CfD Capacity (MW)
2007,East Anglia 3,1200,Planning Permission Granted,Awaiting Construction,NaN
2030,Hornsea Project Two - Optimus and Breesea,1400,Planning Permission Granted,Awaiting Construction,NaN
2044,Dogger Bank Teesside A,1200,Planning Permission Granted,Awaiting Construction,NaN
2048,Neart na Gaoithe,450,Planning Permission Granted,Awaiting Construction,448
2053,Inch Cape,784,Planning Permission Granted,Awaiting Construction,NaN
2055,Seagreen Alpha,525,Planning Permission Granted,Awaiting Construction,NaN
2056,Triton Knoll,860,Planning Permission Granted,Awaiting Construction,NaN
2058,MacColl Offshore Windfarm (Moray Firth),317,Planning Permission Granted,Awaiting Construction,NaN
2060,Telford Offshore Windfarm (Moray Firth),317,Planning Permission Granted,Awaiting Construction,NaN
2061,Stevenson Offshore Windfarm (Moray Firth),316,Planning Permission Granted,Awaiting Construction,NaN


In [18]:
df['Development Status'].unique()

array([u'Abandoned', u'Planning Permission Refused',
       u'Planning Application Withdrawn', u'Operational',
       u'Appeal Refused', u'Appeal Withdrawn',
       u'Planning Permission Expired', u'Planning Permission Granted',
       u'Under Construction', u'No Application Required',
       u'Decommissioned', u'Appeal Granted',
       u'Planning Application Submitted', u'Secretary of State - Granted',
       u'Appeal Lodged', u'Secretary of State - Refusal',
       u'Planning permission granted'], dtype=object)

In [22]:
df.columns

Index([u'Old Ref ID', u'Ref ID', u'Record Last Updated (dd/mm/yyyy)',
       u'Operator (or Applicant)', u'Site Name', u'Technology Type',
       u'Installed Capacity (MWelec)', u'CHP Enabled', u'RO Banding (ROC/MWh)',
       u'FiT Tariff (p/kWh)', u'CfD Capacity (MW)', u'Turbine Capacity (MW)',
       u'No. of Turbines', u'Height of Turbines (m)',
       u'Mounting Type for Solar', u'Development Status',
       u'Development Status (short)', u'Address', u'County', u'Region',
       u'Country', u'Post Code', u'X-coordinate', u'Y-coordinate',
       u'Planning Authority', u'Planning Application Reference',
       u'Appeal Reference', u'Secretary of State Reference',
       u'Type of Secretary of State Intervention', u'Judicial Review',
       u'Offshore Wind Round', u'Planning Application Submitted',
       u'Planning Application Withdrawn', u'Planning Permission Refused',
       u'Appeal Lodged', u'Appeal Withdrawn', u'Appeal Refused',
       u'Appeal Granted', u'Planning Permission Gr

In [9]:
df['Technology Type'].unique()

array([u'Biomass (dedicated)', u'Advanced Conversion Technologies',
       u'Anaerobic Digestion', u'EfW Incineration', u'Biomass (co-firing)',
       u'Large Hydro', u'Small Hydro', u'Landfill Gas',
       u'Solar Photovoltaics', u'Sewage Sludge Digestion',
       u'Tidal Barrage and Tidal Stream', u'Shoreline Wave',
       u'Wind Offshore', u'Wind Onshore', u'Hot Dry Rocks (HDR)',
       u'Pumped Storage Hydroelectricity', u'Liquid Air Energy Storage',
       u'Battery', u'Flywheels', u'Fuel Cell (Hydrogen)'], dtype=object)

In [20]:
waste = df.loc[(df['Technology Type']=='EfW Incineration')|(df['Technology Type']=='Advanced Conversion Technologies')]
waste['year'] = waste['Operational'].dt.year
waste['month'] = waste['Operational'].dt.month
waste['appyear'] = waste['Planning Application Submitted'].dt.year
waste['appmonth'] = waste['Planning Application Submitted'].dt.month
#onapp[['Installed Capacity (MWelec)','year','month']].groupby(['year','month']).sum().reset_index().to_csv('onapp.csv')
waste[['Installed Capacity (MWelec)','Development Status','Technology Type']].groupby(['Technology Type','Development Status']).agg({'Installed Capacity (MWelec)': ['sum','count']}).reset_index()
waste.to_csv('waste.csv',index=False, encoding='utf-8')

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [12]:
operational.sort_values(by='Installed Capacity (MWelec)')

4,Site Name,Technology Type,Installed Capacity (MWelec),Region,lat,lon,Year Open,CHP Enabled,Operator (or Applicant),Fuel,Type
5203,Flat Holm Microgrid Project,Battery,0.005,Wales,51.378056,-3.122498,2006,n/a,Wind & Sun,Storage,Battery
5245,JB Wheaton,Battery,0.005,South West,50.840997,-2.935433,2016,n/a,E.On,Storage,Battery
5207,Horse Island Microgrid Project,Battery,0.012,Scotland,57.987532,-5.344452,2009,n/a,Wind & Sun,Storage,Battery
5204,Foula Community Electricity Scheme,Battery,0.016,Scotland,60.135006,-2.054350,2008,n/a,Foula Community Electricity Trust,Storage,Battery
5210,Isle of Rum Microgrid Project,Battery,0.045,Scotland,57.009477,-6.323878,2015,n/a,Wind & Sun,Storage,Battery
5209,Isle of Muck Microgrid System,Battery,0.045,Scotland,56.835812,-6.238438,2013,n/a,Wind & Sun,Storage,Battery
5237,The Everest System,Battery,0.050,Eastern,52.561638,1.180018,2016,n/a,Connected Energy,Storage,Battery
5208,Isle of Eigg Electrification Project,Battery,0.060,Scotland,56.899526,-6.141617,2008,n/a,Eigg Electric,Storage,Battery
5240,Western International Market (WIM) Site,Battery,0.240,London,51.496154,-0.408174,2016,n/a,London Borough of Hounslow,Storage,Battery
5234,Limberlost Solar Farm,Battery,0.250,South East,51.381113,-1.232508,2015,n/a,Anesco,Storage,Battery


In [18]:
#df[df['Post Code'].str.contains('SN16')]
df[(df['Technology Type']=='Photovoltaic')&(df['Installed Capacity (MWelec)']<5)&(df['Operational'].dt.year==2011)]

4,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,...,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,lat,lon


In [27]:
df[['Site Name','Post Code','Installed Capacity (MWelec)','County','X-coordinate','Operational']][(df['County']=='Wiltshire')&(df.Operational.dt.year==2011)]

4,Site Name,Post Code,Installed Capacity (MWelec),County,X-coordinate,Operational
968,Malmesbury Solar,SN16 9SR,5.0,Wiltshire,392558.0,2011-07-25 00:00:00
1002,Kingston Farm,BA15 1AH,5.0,Wiltshire,383092.0,2011-10-01 00:00:00
1134,Lake Farm,SN15 4SQ,5.0,Wiltshire,393800.0,2011-07-01 00:00:00


In [22]:
df.columns

Index([                             u'Old Ref ID',
                                        u'Ref ID',
              u'Record Last Updated (dd/mm/yyyy)',
                       u'Operator (or Applicant)',
                                     u'Site Name',
                               u'Technology Type',
                                  u'Storage Type',
               u'Storage Co-location REPD Ref ID',
                   u'Installed Capacity (MWelec)',
                                   u'CHP Enabled',
                          u'RO Banding (ROC/MWh)',
                            u'FiT Tariff (p/kWh)',
                             u'CfD Capacity (MW)',
                         u'Turbine Capacity (MW)',
                               u'No. of Turbines',
                        u'Height of Turbines (m)',
                       u'Mounting Type for Solar',
                            u'Development Status',
                    u'Development Status (short)',
                               

In [28]:
df['Installed Capacity (MWelec)'][(df['Technology Type']=='Wind Onshore')&(df['Development Status']=='Operational')].sum()

12817.891999999994

In [30]:
df[(df['Technology Type']=='Wind Onshore')&(df['Development Status']=='Operational')].sort_values(by='Installed Capacity (MWelec)')

4,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,...,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,lat,lon
2952,A0355,3723,2015-01-07,Green Cat Renewables,Yonderton Farm,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2010-07-23 00:00:00,NaN,NaN,NaN,NaN,2014-08-01 00:00:00,2014-12-01 00:00:00,57.430837,-1.941896
3317,EN00443,4113,2014-12-02,Wind Direct,East Midlands Airport,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2008-03-28 00:00:00,NaN,NaN,NaN,NaN,2011-04-16 00:00:00,2011-05-10 00:00:00,52.826198,-1.342652
3132,C1356,3924,2015-11-25,The Farm Energy Partnership LLP,Bloodhills - repowering,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2013-05-14 00:00:00,NaN,NaN,NaN,NaN,2014-07-15 00:00:00,2015-06-15 00:00:00,52.706978,1.664421
3126,C0862,3918,2016-04-22,George Richards Farms Partnership,Goonabarn Farm Wind Turbines,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2013-07-04 00:00:00,NaN,NaN,NaN,NaN,2016-01-01 00:00:00,2016-04-01 00:00:00,50.349671,-4.960596
2909,A0109,3678,2016-07-28,Tobermore Concrete Products,Magherafelt,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2010-11-11 00:00:00,NaN,NaN,NaN,2015-11-11 00:00:00,2015-11-01 00:00:00,2015-06-12 00:00:00,54.769045,-6.647716
3683,H0047,4514,2016-10-26,Sunnyside Wind Farm,Sunnyside wind cluster,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2013-12-19 00:00:00,NaN,NaN,NaN,NaN,2015-06-15 00:00:00,2015-11-15 00:00:00,55.379019,-3.948910
2704,C1658,3429,2018-11-19,Urban Wind,Moorhouse Farm,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2014-09-11 00:00:00,NaN,NaN,NaN,2017-09-11 00:00:00,2016-08-15 00:00:00,2018-05-15 00:00:00,55.903903,-2.250838
2653,C1509,3361,2017-02-23,Private Developer,PROMISED LAND FARM,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2013-10-15 00:00:00,NaN,NaN,NaN,NaN,2014-09-22 00:00:00,2014-10-22 00:00:00,53.511331,-2.737805
2629,C1708,3321,2015-01-05,Windflow UK,New Holland,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2013-11-06 00:00:00,NaN,NaN,NaN,NaN,NaN,2014-10-20 00:00:00,58.913298,-2.868907
2588,N00422W,3272,2010-03-03,Private Developer,Lodge Farm 1,Wind Onshore,NaN,NaN,1.00,n/a,...,NaN,2005-10-01 00:00:00,NaN,NaN,NaN,NaN,NaN,2006-08-01 00:00:00,51.726298,-5.060933


In [33]:
df['Installed Capacity (MWelec)'][(df['Technology Type']=='Solar Photovoltaics')&(df['Development Status']=='Operational')].sum()b

8380.88999999999

In [32]:
df['Technology Type'].unique()

array([u'Biomass (dedicated)', u'Advanced Conversion Technologies',
       u'Anaerobic Digestion', u'EfW Incineration', u'Biomass (co-firing)',
       u'Large Hydro', u'Small Hydro', u'Landfill Gas',
       u'Solar Photovoltaics', u'Sewage Sludge Digestion',
       u'Tidal Barrage and Tidal Stream', u'Shoreline Wave',
       u'Wind Offshore', u'Wind Onshore', u'Hot Dry Rocks (HDR)',
       u'Pumped Storage Hydroelectricity', u'Liquid Air Energy Storage',
       u'Battery', u'Flywheels', u'Fuel Cell (Hydrogen)'], dtype=object)

In [38]:
df[(df['Technology Type']=='Battery')&(df['Site Name'].str.contains('Fair'))]







4,Old Ref ID,Ref ID,Record Last Updated (dd/mm/yyyy),Operator (or Applicant),Site Name,Technology Type,Storage Type,Storage Co-location REPD Ref ID,Installed Capacity (MWelec),CHP Enabled,...,Appeal Granted,Planning Permission Granted,Secretary of State - Intervened,Secretary of State - Refusal,Secretary of State - Granted,Planning Permission Expired,Under Construction,Operational,lat,lon
5393,NaN,7091,2018-10-19,Renewable Energy Systems,Fair Oak Lane,Battery,Stand-alone Storage,n/a,49.9,n/a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.341536,-0.320369


In [114]:
renewable[renewable.Fuel=='Biomass'].sort_values(['Capacity (MW)'],ascending=False)

,CHP,Capacity (MW),Company,Fuel,Latitude,Longitude,Region,Site,Type,Year Open,yearEnd,yearStart
170,No,645.000,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2014.0,2019,2014
5134,n/a,645.000,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2018.0,2019,2018
154,No,645.000,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2017.0,2019,2017
153,No,645.000,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,Co-firing,2013.0,2019,2013
96,No,420.000,Lynemouth Power,Biomass,55.206722,-1.532504,North East,Lynemouth Power Station,NaN,2018.0,2019,2018
133,Yes,299.000,Macquarie/ PKA/ MGT Power,Biomass,54.602146,-1.161282,North East,Tees Renewable Energy Plant,NaN,NaN,2019,2019
135,Yes,65.000,RWE,Biomass,56.201180,-3.162255,Scotland,Markinch Biomass CHP Plant,NaN,2014.0,2019,2014
174,No,50.400,E.ON (formerly Powergen). Agent Environmental...,Biomass,55.152401,-3.380891,Scotland,Stevens Croft,NaN,2007.0,2019,2007
39,Yes,49.000,Iggesund Paperboard (Workington),Biomass,54.663976,-3.547331,North West,Iggesund Paperboard CHP Plant,NaN,2013.0,2019,2013
163,No,44.200,Burmeister & Wain Scandinavian Contractor A/S ...,Biomass,52.479148,0.954434,Eastern,Snetterton Biomass Plant,NaN,2016.0,2019,2016


In [115]:
renewable[renewable.Site.str.contains('Drax')]

,CHP,Capacity (MW),Company,Fuel,Latitude,Longitude,Region,Site,Type,Year Open,yearEnd,yearStart
153,No,645.0,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,Co-firing,2013.0,2019,2013
154,No,645.0,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2017.0,2019,2017
170,No,645.0,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2014.0,2019,2014
5134,n/a,645.0,Drax Power,Biomass,53.735589,-0.991097,Yorkshire and Humber,Drax Biomass,NaN,2018.0,2019,2018
